# Single Subject Segmentation Q1K

In [1]:
import q1k_sync_tools as qst
import mne
mne.viz.set_browser_backend('qt')
import pandas as pd
pd.options.mode.chained_assignment = None
import pylossless as ll
import numpy as np

Using qt as 2D backend.


In [2]:
subject_raw_eeg = '/home/james/q1k/as/sourcedata/s06_v2/s06_v2_eeg/s06_v2_as_20230404_014928.mff'
subject_raw_et = '/home/james/q1k/as/sourcedata/s06_v2/s06_v2_eyetracking/s06v2_as.asc'
subject_derivative = './derivatives/pylossless/sub-006/eeg/sub-006_task-as_eeg.edf'
task_code = 'as'

Reading EGI MFF Header from /home/james/q1k/as/sourcedata/s06_v2/s06_v2_eeg/s06_v2_as_20230404_014928.mff...
    Reading events ...
    Assembling measurement info ...
    Synthesizing trigger channel "STI 014" ...
    Excluding events {} ...


In [3]:
# Load EEG recording
eeg_raw = mne.io.read_raw_egi(subject_raw_eeg)

# Get event info from source recording
eeg_events = mne.find_events(eeg_raw, shortest_event=1)
eeg_event_dict = qst.get_event_dict(eeg_raw, eeg_events)

# Infer new events and labels from EEG paradigm procedure
eeg_events, eeg_stims, eeg_iti, eeg_event_dict = qst.eeg_event_test(eeg_events, eeg_event_dict, task_name=task_code)

# Read eyetracking data and extact events
et_raw = mne.io.read_raw_eyelink(subject_raw_et)
et_annot_events, et_annot_event_dict = mne.events_from_annotations(et_raw)
et_raw_df = et_raw.to_data_frame()

# Infer new events from paradigm procedure
et_event_raw_df, et_events, et_stims, et_iti = qst.et_event_test(et_raw_df, task_name=task_code)

# Meld both ET and EEG together into new raw structure
eeg_et_raw = qst.eeg_et_combine(eeg_raw, et_raw, eeg_stims, et_stims)

# Using the melded raw, build final event objects
eeg_et_events = mne.find_events(eeg_et_raw, shortest_event=1)
eeg_et_event_dict = qst.get_event_dict(eeg_et_raw, eeg_et_events)

# Free up memory
del et_event_raw_df
del et_events
del et_stims
del et_iti
del et_raw_df
del et_annot_events
del et_annot_event_dict
del eeg_events
del eeg_stims
del eeg_iti
del eeg_event_dict

print('Done!')

781 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27]
Number of stimulus onset DIN events: 52
Loading /home/james/q1k/as/sourcedata/s06_v2/s06_v2_eyetracking/s06v2_as.asc
Pixel coordinate data detected.
Pass `scalings=dict(eyegaze=1e3)` when using plot method to make traces more legible.
Pupil-size area reported.
There are 52 recording blocks in this file. Times between blocks will be annotated with bad_rec_gap.
Used Annotations descriptions: ['ANTI-SACCADE_TRIAL', 'CORRECTIVE-SACCADE_TRIAL', 'DISPLAY_FIXATION', 'DISTRACTOR_OFFSET', 'GAZE_TO_DISTRACTOR', 'GAZE_TO_FIX', 'GAZE_TO_TARGET', 'INVALID_TRIAL', 'ONSET_DISTRACTOR', 'ONSET_REWARD', 'PREDICT_TARGET_GAZE', 'PRO-SACCADE_TRIAL', 'TARGET_ONSET', 'TRACKER_TIME 2 2977896.278', 'TRACKER_TIME 3 3037899.929', 'TRACKER_TIME 4 3097924.93', 'TRACKER_TIME 5 3157959.299', 'TRIAL_END', 'blink_R', 'fixation_R', 'saccade_R']
Number of eye-tracking stimulus onset DIN events: 52
Zero order c

In [4]:
# Build initial state; nothing applied yet
ll_state = ll.LosslessPipeline()
ll_state = ll_state.load_ll_derivative(subject_derivative)

Extracting EDF parameters from /home/james/q1k/pilot/q1k_external_pilot/derivatives/pylossless/sub-006/eeg/sub-006_task-as_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from derivatives/pylossless/sub-006/eeg/sub-006_task-as_events.tsv.
Reading channel info from derivatives/pylossless/sub-006/eeg/sub-006_task-as_channels.tsv.
Reading electrode coords from derivatives/pylossless/sub-006/eeg/sub-006_space-CapTrak_electrodes.tsv.
Reading derivatives/pylossless/sub-006/eeg/sub-006_task-as_ica1_ica.fif ...
Now restoring ICA solution ...
Ready.
Reading derivatives/pylossless/sub-006/eeg/sub-006_task-as_ica2_ica.fif ...
Now restoring ICA solution ...
Ready.


/home/james/q1k/pilot/q1k_external_pilot/eeg-env/lib/python3.8/site-packages/pylossless/pipeline.py:1118: RuntimeWarning: The number of channels in the channels.tsv sidecar file (157) does not match the number of channels in the raw data file (129). Will not try to set channel names.
  self.raw = mne_bids.read_raw_bids(derivatives_path)
/home/james/q1k/pilot/q1k_external_pilot/eeg-env/lib/python3.8/site-packages/pylossless/pipeline.py:1118: RuntimeWarning: Cannot set channel type for the following channels, as they are missing in the raw data: DIN2, DIN3, DIN4, STI 014, TRSP, TSYN, VBeg, bgn1, dbgl, dbgr, ddtl, ddtr, dfxl, dfxr, drwl, drwr, dtgl, dtgr, gbdl, gbdr, gbtl, gbtr, gddl, gddr, gfxl, gfxr, gttl, gttr
  self.raw = mne_bids.read_raw_bids(derivatives_path)


In [5]:
# Plot channel data only
mne.viz.plot_raw(ll_state.raw)

Channels marked as bad:
['E125', 'E126', 'E127', 'E128']


In [6]:
%matplotlib qt
# Plot comonents
ll_state.ica2.plot_sources(ll_state.raw)

Creating RawArray with float64 data, n_channels=117, n_times=574000
    Range : 0 ... 573999 =      0.000 ...   573.999 secs
Ready.


In [7]:
%matplotlib qt
# Plot topos
ll_state.ica2.plot_components(inst=ll_state.raw)

[<MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 20 Axes>,
 <MNEFigure size 975x967 with 17 Axes>]

    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
251 matching events found
No baseline correction applied
0 projection items activated


In [8]:
# Merge marks down to bads (aka manual)
ll_qcr = ll_state.raw.copy()
manual = []
for flag_type in ll_state.flags['ch']:
    manual.extend(ll_state.flags['ch'][flag_type])
print(ll_state.flags['ch'])
ll_qcr.info['bads'].extend(manual)
ll_qcr.info['bads'] = list(set(ll_qcr.info['bads']))

# Go read the ICLabel info from file and add to exclude
df = pd.read_csv(subject_derivative.replace('_eeg.edf', '_iclabels.tsv'), sep='\t')
ll_state.ica2.exclude = list(df[df['ic_type'].str.match('eog|muscle|ch_noise|ecg')].index)

# Load the data and apply the ICA
ll_qcr.load_data()
ll_state.ica2.apply(ll_qcr)
ll_qcr = ll_qcr.filter(l_freq=1.0, h_freq=40.0)
ll_qcr = ll_qcr.set_eeg_reference(ref_channels="average")
ll_qcr

{'ch_sd': array(['E49'], dtype=object), 'rank': array(['E50'], dtype=object)}
Reading 0 ... 573999  =      0.000 ...   573.999 secs...
Applying ICA to Raw instance
    Transforming to ICA space (117 components)
    Zeroing out 22 ICA components
    Projecting back using 123 PCA components
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 3301 samples (3.301 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


[Parallel(n_jobs=1)]: Done 129 out of 129 | elapsed:    2.2s finished


<RawEDF | sub-006_task-as_eeg.edf, 129 x 574000 (574.0 s), ~565.1 MB, data loaded>

In [15]:
# Plot cleaned data
mne.viz.plot_raw(ll_qcr, events=eeg_et_events, event_id=eeg_et_event_dict)

Channels marked as bad:
['E128', 'E49', 'E126', 'E125', 'E127', 'E50']


In [11]:
# Relabel condition vars for niceness
eeg_et_event_dict['distractor/ontime/gddl'] = eeg_et_event_dict.pop('gddl')
eeg_et_event_dict['distractor/ontime/gddr'] = eeg_et_event_dict.pop('gddr')
eeg_et_event_dict['distractor/late/gbdr'] = eeg_et_event_dict.pop('gbdr')
eeg_et_event_dict['distractor/late/gbdl'] = eeg_et_event_dict.pop('gbdl')

eeg_et_event_dict['target/ontime/gddl'] = eeg_et_event_dict.pop('gttl')
eeg_et_event_dict['target/ontime/gddr'] = eeg_et_event_dict.pop('gttr')
eeg_et_event_dict['target/early/gbtr'] = eeg_et_event_dict.pop('gbtr')
eeg_et_event_dict['target/early/gbtl'] = eeg_et_event_dict.pop('gbtl')

eeg_et_event_dict

{'DIN': 1,
 'TSYN': 2,
 'bgn1': 3,
 'dfxr': 4,
 'gfxr': 5,
 'ddtr': 6,
 'dbgr': 7,
 'dtgr': 9,
 'drwr': 11,
 'TRSP': 12,
 'dfxl': 15,
 'gfxl': 16,
 'ddtl': 17,
 'dbgl': 18,
 'dtgl': 20,
 'drwl': 22,
 'DIN3': 25,
 'DIN4': 26,
 'DIN2': 27,
 'VBeg': 28,
 'distractor/ontime/gddl': 24,
 'distractor/ontime/gddr': 14,
 'distractor/late/gbdr': 8,
 'distractor/late/gbdl': 19,
 'target/ontime/gddl': 21,
 'target/ontime/gddr': 10,
 'target/early/gbtr': 13,
 'target/early/gbtl': 23}

In [12]:
# Epoch structure is created for ALL events, then you 'segment' by indexing into it
epochs = mne.Epochs(ll_qcr, eeg_et_events, event_id=eeg_et_event_dict, tmin=-0.5, tmax=1.0, on_missing='warn')
epochs

Not setting metadata
779 matching events found
Setting baseline interval to [-0.5, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated


/tmp/ipykernel_19766/2076716230.py:2: RuntimeWarning: No matching events found for DIN2 (event id 27)
  epochs = mne.Epochs(ll_qcr, eeg_et_events, event_id=eeg_et_event_dict, tmin=-0.5, tmax=1.0, on_missing='warn')
/tmp/ipykernel_19766/2076716230.py:2: RuntimeWarning: No matching events found for VBeg (event id 28)
  epochs = mne.Epochs(ll_qcr, eeg_et_events, event_id=eeg_et_event_dict, tmin=-0.5, tmax=1.0, on_missing='warn')


Number of events,779
Events,DIN: 3DIN2: 0DIN3: 106DIN4: 72TRSP: 12TSYN: 52VBeg: 0bgn1: 26dbgl: 25dbgr: 26ddtl: 26ddtr: 26dfxl: 26dfxr: 26distractor/late/gbdl: 6distractor/late/gbdr: 14distractor/ontime/gddl: 104distractor/ontime/gddr: 26drwl: 20drwr: 52dtgl: 5dtgr: 12gfxl: 26gfxr: 26target/early/gbtl: 6target/early/gbtr: 4target/ontime/gddl: 26target/ontime/gddr: 26
Time range,-0.500 – 1.000 s
Baseline,-0.500 – 0.000 s


In [13]:
# Check trial counts
epochs[['distractor', 'target']]

Number of events,212
Events,distractor/late/gbdl: 6distractor/late/gbdr: 14distractor/ontime/gddl: 104distractor/ontime/gddr: 26target/early/gbtl: 6target/early/gbtr: 4target/ontime/gddl: 26target/ontime/gddr: 26
Time range,-0.500 – 1.000 s
Baseline,-0.500 – 0.000 s


In [18]:
# Plot ERPs
evokeds = {'distractor': epochs['distractor'].average(), 'target': epochs['target'].average()}
# evokeds = {'distractor': list(epochs['distractor'].iter_evoked()), 'target': list(epochs['target'].iter_evoked())}
mne.viz.plot_compare_evokeds(evokeds, picks=['E6'], combine='mean')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
combining channels using "mean"
combining channels using "mean"


/tmp/ipykernel_19766/134790767.py:4: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  mne.viz.plot_compare_evokeds(evokeds, picks=['E6'], combine='mean')


[<Figure size 800x600 with 2 Axes>]